# Train a model with Episodic Training
Episodic training has attracted a lot of interest in the early years of Few-Shot Learning research. Some papers still use it, and refer to it as "meta-learning".

Recent works distinguish the Few-Shot Classifier from the training framework, so as from v1.0 of EasyFSL, methods to episodically train a classifier were taken out of the logic of the FewShotClassifier class. Instead, we provide in this notebook an example of how to perform episodic training on a few-shot classifier.

Use it, copy it, change it, get crazy.

## Getting started
First we're going to do some imports (this is not the interesting part).

In [1]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [2]:
!pip install easyfsl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 1.0 MB/s eta 0:00:00


In [3]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import GTSRB
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

Then we're gonna do the most important thing in Machine Learning research: ensuring reproducibility by setting the random seed. We're going to set the seed for all random packages that we could possibly use, plus some other stuff to make CUDA deterministic (see [here](https://pytorch.org/docs/stable/notes/randomness.html)).

I strongly encourage that you do this in **all your scripts**.

In [4]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Then we're gonna set the shape of our problem.

Also we define our set-up, like the device (change it if you don't have CUDA) or the number of workers for data loading.

In [5]:
n_way = 5
n_shot = 5
n_query = 10

DEVICE = "cuda"
n_workers = 12

## Training

First we define our data loaders for training and validation. You can see that I chose tu use CUB in this notebook, because it's a small dataset, so we can have good results quite quickly. We use `CUB` and `TaskSampler` which are built-in objects from EasyFSL.

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from urllib.request import urlretrieve
import zipfile
import time
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from torchvision.datasets import GTSRB
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100
image_size = 28

# Instantiate the datasets
train_set = GTSRB(
    root="./data",
    # background=True,
    split="train",
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
    ),
    download=True,
)

separate = int(np.shape(train_set)[0] * 0.8)
train_set, valid_set = torch.utils.data.random_split(train_set, [separate, np.shape(train_set)[0] - separate])

train_set.get_labels = lambda: [instance[1] for instance in train_set]

valid_set.get_labels = lambda: [instance[1] for instance in valid_set]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)

val_sampler = TaskSampler(
    valid_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

val_loader = DataLoader(
    valid_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

100%|██████████| 187490228/187490228 [00:07<00:00, 24493160.00it/s]


Extracting data/gtsrb/GTSRB-Training_fixed.zip to data/gtsrb


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2007: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: User

In [8]:
!mkdir /content/data/gtsrb/GTSRB/Testing

In [9]:
!mv /content/data/gtsrb/GTSRB/Training/00021 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00037 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00042 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00016 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00006 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00005 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00004 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00035 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00000 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00029 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00010 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00012 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00040 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00009 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00036 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00031 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00041 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00018 /content/data/gtsrb/GTSRB/Testing

In [10]:
import os

old_folder_name = '/content/data/gtsrb/GTSRB/Training'
new_folder_name = '/content/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/content/data/gtsrb/GTSRB/Testing'
new_folder_name = '/content/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [11]:
n_test_tasks = 1000

test_set = GTSRB(
    root="./data",
    # background=False,
    split="train",
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    ),
    download=False,
)

test_set.get_labels = lambda: [instance[1] for instance in test_set]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [12]:
len(train_set), len(valid_set), len(test_set)

(21312, 5328, 10680)

In [ ]:
# !wget "http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip"
# !wget "https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip"

--2023-05-20 13:18:24--  http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.168, 2001:67c:10ec:36c2::168
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.168|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip [following]
--2023-05-20 13:18:24--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174298785 (166M) [application/zip]
Saving to: ‘BelgiumTSC_Training.zip’

BelgiumTSC_Training 100%[===================>] 166.22M  8.65MB/s    in 20s     

2023-05-20 13:18:45 (8.38 MB/s) - ‘BelgiumTSC_Training.zip’ saved [174298785/174298785]

--2023-05-20 13:18:45--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.168, 

In [13]:
import shutil
folder_path = "/content/data/gtsrb/GTSRB/Training"
shutil.rmtree(folder_path)
# folder_path = "/content/data/gtsrb/GTSRB/Testing"
# shutil.rmtree(folder_path)

In [ ]:
import os

old_folder_name = '/content/data/gtsrb/GTSRB/Training'
new_folder_name = '/content/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [14]:
extract_path = "./data/gtsrb/GTSRB"
validation_data_dir = './data/gtsrb/GTSRB/Training'

def train_data(train_url):
    zip_dir = "./data/BelgiumTSC_Training.zip"

    print("Downloading Belgium TSC Training Dataset\n")
    urlretrieve(train_url, zip_dir)
    zip_ref = zipfile.ZipFile(zip_dir)

    print("Extracting Zip\n")
    zip_ref.extractall(extract_path)
    zip_ref.close()

def test_data(test_url):
    zip_dir = "./data/BelgiumTSC_Training.zip"

    print("Downloading Belgium TSC Testing Dataset\n")
    urlretrieve(test_url, zip_dir)
    zip_ref = zipfile.ZipFile(zip_dir)

    print("Extracting Zip\n")
    zip_ref.extractall(extract_path)
    zip_ref.close()


def download_data():
    print("Download Datasets")
    start = time.time()
    train_data("http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip")
    # test_data("http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip")
    end = time.time()
    print("Downloading Datasets 'BelgiumTSC' took ", end - start, 'seconds')

download_data()

Download Datasets

Extracting Zip



In [ ]:
import os
import shutil

cntt =0
summ = 0
folder_A_path = '/content/data/gtsrb/GTSRB/Training'

# Iterate over subdirectories in folder_A
for root, dirs, files in os.walk(folder_A_path):
    for dir in dirs:
        folder_B_path = os.path.join(root, dir)
        ppm_file_count = sum(1 for file in os.listdir(folder_B_path) if file.endswith('.ppm'))
        print(f"Folder {folder_B_path} contains {ppm_file_count} .ppm files.")

        summ += ppm_file_count

        if ppm_file_count < 80:
          cntt += 1
          shutil.rmtree(folder_B_path)

print(summ, "SUM")

In [ ]:
len(valid_set) + len(train_set)

4575

In [ ]:
import random
from typing import Dict, Iterator, List, Tuple, Union

import torch
from torch import Tensor
from torch.utils.data import Sampler

from easyfsl.datasets import FewShotDataset

GENERIC_TYPING_ERROR_MESSAGE = (
    "Check out the output's type of your dataset's __getitem__() method."
    "It must be a Tuple[Tensor, int] or Tuple[Tensor, 0-dim Tensor]."
)

print("HAHA2")
class TaskSamplerr(Sampler):
    """
    Samples batches in the shape of few-shot classification tasks. At each iteration, it will sample
    n_way classes, and then sample support and query images from these classes.
    """

    def __init__(
        self,
        dataset: FewShotDataset,
        n_way: int,
        n_shot: int,
        n_query: int,
        n_tasks: int,
    ):
        """
        Args:
            dataset: dataset from which to sample classification tasks. Must have implement get_labels() from
                FewShotDataset.
            n_way: number of classes in one task
            n_shot: number of support images for each class in one task
            n_query: number of query images for each class in one task
            n_tasks: number of tasks to sample
        """
        print("HAHAAA: ", dataset.get_labels())
        super().__init__(data_source=None)
        self.n_way = n_way
        self.n_shot = n_shot
        self.n_query = n_query
        self.n_tasks = n_tasks

        self.items_per_label: Dict[int, List[int]] = {}
        
        for item, label in enumerate(dataset.get_labels()):
            print("HAHA3: ", item, label)
            if label in self.items_per_label:
                self.items_per_label[label].append(item)
            else:
                self.items_per_label[label] = [item]

        print("HAHA: ", self.items_per_label) 

        for itemss in self.items_per_label:
          print(itemss, len(self.items_per_label[itemss]))

        self._check_dataset_size_fits_sampler_parameters()

    def __len__(self) -> int:
        return self.n_tasks

    def __iter__(self) -> Iterator[List[int]]:
        """
        Sample n_way labels uniformly at random,
        and then sample n_shot + n_query items for each label, also uniformly at random.
        Yields:
            a list of indices of length (n_way * (n_shot + n_query))
        """
        for _ in range(self.n_tasks):
            yield torch.cat(
                [
                    torch.tensor(
                        random.sample(
                            self.items_per_label[label], self.n_shot + self.n_query
                        )
                    )
                    for label in random.sample(
                        sorted(self.items_per_label.keys()), self.n_way
                    )
                ]
            ).tolist()

    def episodic_collate_fn(
        self, input_data: List[Tuple[Tensor, Union[Tensor, int]]]
    ) -> Tuple[Tensor, Tensor, Tensor, Tensor, List[int]]:
        """
        Collate function to be used as argument for the collate_fn parameter of episodic
            data loaders.
        Args:
            input_data: each element is a tuple containing:
                - an image as a torch Tensor of shape (n_channels, height, width)
                - the label of this image as an int or a 0-dim tensor
        Returns:
            tuple(Tensor, Tensor, Tensor, Tensor, list[int]): respectively:
                - support images of shape (n_way * n_shot, n_channels, height, width),
                - their labels of shape (n_way * n_shot),
                - query images of shape (n_way * n_query, n_channels, height, width)
                - their labels of shape (n_way * n_query),
                - the dataset class ids of the class sampled in the episode
        """
        input_data_with_int_labels = self._cast_input_data_to_tensor_int_tuple(
            input_data
        )
        true_class_ids = list({x[1] for x in input_data_with_int_labels})
        all_images = torch.cat([x[0].unsqueeze(0) for x in input_data_with_int_labels])
        all_images = all_images.reshape(
            (self.n_way, self.n_shot + self.n_query, *all_images.shape[1:])
        )
        all_labels = torch.tensor(
            [true_class_ids.index(x[1]) for x in input_data_with_int_labels]
        ).reshape((self.n_way, self.n_shot + self.n_query))
        support_images = all_images[:, : self.n_shot].reshape(
            (-1, *all_images.shape[2:])
        )
        query_images = all_images[:, self.n_shot :].reshape((-1, *all_images.shape[2:]))
        support_labels = all_labels[:, : self.n_shot].flatten()
        query_labels = all_labels[:, self.n_shot :].flatten()
        return (
            support_images,
            support_labels,
            query_images,
            query_labels,
            true_class_ids,
        )

    @staticmethod
    def _cast_input_data_to_tensor_int_tuple(
        input_data: List[Tuple[Tensor, Union[Tensor, int]]]
    ) -> List[Tuple[Tensor, int]]:
        """
        Check the type of the input for the episodic_collate_fn method, and cast it to the right type if possible.
        Args:
            input_data: each element is a tuple containing:
                - an image as a torch Tensor of shape (n_channels, height, width)
                - the label of this image as an int or a 0-dim tensor
        Returns:
            the input data with the labels cast to int
        Raises:
            TypeError : Wrong type of input images or labels
            ValueError: Input label is not a 0-dim tensor
        """
        for image, label in input_data:
            if not isinstance(image, Tensor):
                raise TypeError(
                    f"Illegal type of input instance: {type(image)}. "
                    + GENERIC_TYPING_ERROR_MESSAGE
                )
            if not isinstance(label, int):
                if not isinstance(label, Tensor):
                    raise TypeError(
                        f"Illegal type of input label: {type(label)}. "
                        + GENERIC_TYPING_ERROR_MESSAGE
                    )
                if label.dtype not in {
                    torch.uint8,
                    torch.int8,
                    torch.int16,
                    torch.int32,
                    torch.int64,
                }:
                    raise TypeError(
                        f"Illegal dtype of input label tensor: {label.dtype}. "
                        + GENERIC_TYPING_ERROR_MESSAGE
                    )
                if label.ndim != 0:
                    raise ValueError(
                        f"Illegal shape for input label tensor: {label.shape}. "
                        + GENERIC_TYPING_ERROR_MESSAGE
                    )

        return [(image, int(label)) for (image, label) in input_data]

    def _check_dataset_size_fits_sampler_parameters(self):
        """
        Check that the dataset size is compatible with the sampler parameters
        """
        self._check_dataset_has_enough_labels()
        self._check_dataset_has_enough_items_per_label()

    def _check_dataset_has_enough_labels(self):
        if self.n_way > len(self.items_per_label):
            raise ValueError(
                f"The number of labels in the dataset ({len(self.items_per_label)} "
                f"must be greater or equal to n_way ({self.n_way})."
            )

    def _check_dataset_has_enough_items_per_label(self):
        print("HI")
        number_of_samples_per_label = [
            len(items_for_label) for items_for_label in self.items_per_label.values()
        ]
        print("SAMPLES: ", number_of_samples_per_label)
        minimum_number_of_samples_per_label = min(number_of_samples_per_label)
        print("MIN: ", minimum_number_of_samples_per_label)
        label_with_minimum_number_of_samples = number_of_samples_per_label.index(
            minimum_number_of_samples_per_label
        )

        print("LABEL: ", label_with_minimum_number_of_samples)
        if self.n_shot + self.n_query > minimum_number_of_samples_per_label:
            raise ValueError(
                f"Label {label_with_minimum_number_of_samples} has only {minimum_number_of_samples_per_label} samples"
                f"but all classes must have at least n_shot + n_query ({self.n_shot + self.n_query}) samples."
            )


HAHA2


In [ ]:
import random
from torch.utils.data import Dataset, Subset

def uniform_random_split(TRAIN_SET):
  from torch.utils.data import random_split

  # Assuming you have a dataset called TRAIN_SET
  total_samples = len(TRAIN_SET)

  # Create a list of classes and their respective counts
  class_counts = {c: 0 for _, c in TRAIN_SET}

  # Iterate over the TRAIN_SET and count the samples for each class
  for _, c in TRAIN_SET:
      class_counts[c] += 1

  # Determine the minimum number of samples per class in each set
  min_samples_per_class = 15

  # Calculate the number of samples needed in the validation set for each class
  valid_counts = {c: max(min_samples_per_class - count, 0) for c, count in class_counts.items()}

  # Split the TRAIN_SET into train_set and valid_set
  train_samples = []
  valid_samples = []
  for sample in TRAIN_SET:
      _, c = sample

      # Check if the sample is needed in the validation set for its class
      if valid_counts[c] > 0:
          valid_samples.append(sample)
          valid_counts[c] -= 1
      else:
          train_samples.append(sample)

  # Split the train samples and create train_set and valid_set
  tr_size = int(0.8 * total_samples)
  tst_size = total_samples - tr_size
  train_set, valid_set = random_split(train_samples, [tr_size, tst_size])

  # Concatenate the valid_set with the remaining validation samples
  valid_set = valid_set + valid_samples

  return train_set, valid_set

In [29]:
flag = 0
found = 0

In [31]:
for target_val in range(50, 100, 5):
  import os
  import shutil

  cntt =0
  summ = 0
  folder_A_path = '/content/data/gtsrb/GTSRB/Training'

  # Iterate over subdirectories in folder_A
  for root, dirs, files in os.walk(folder_A_path):
      for dir in dirs:
          folder_B_path = os.path.join(root, dir)
          ppm_file_count = sum(1 for file in os.listdir(folder_B_path) if file.endswith('.ppm'))
          print(f"Folder {folder_B_path} contains {ppm_file_count} .ppm files.")

          summ += ppm_file_count
          print("VAL: ", target_val)
          if ppm_file_count < target_val:
            cntt += 1
            shutil.rmtree(folder_B_path)

  trains_set = GTSRB(
      root="./data",
      # background=True,
      split="train",
      transform=transforms.Compose(
          [
              transforms.Grayscale(num_output_channels=3),
              transforms.RandomResizedCrop(image_size),
              transforms.RandomHorizontalFlip(),
              transforms.ToTensor(),
          ]
      ),
      download=False,
  )

  train_set_size = int(len(trains_set) * 0.8)
  val_set_size = len(trains_set) - train_set_size

  for vall in range(2):
    train_set, valid_set = torch.utils.data.random_split(trains_set, [train_set_size, val_set_size])
    train_set.get_labels = lambda: [instance[1] for instance in train_set]
    valid_set.get_labels = lambda: [instance[1] for instance in valid_set]

    train_dct: dict[int, list[int]] = {}
    for item, label in enumerate(train_set.get_labels()):
      if label in train_dct:
          train_dct[label].append(item)
      else:
          train_dct[label] = [item]
          
    number_of_samples_per_label = [
                len(items_for_label) for items_for_label in train_dct.values()
            ]
    minimum_number_of_samples_per_label = min(number_of_samples_per_label)
    print("MIN: ", minimum_number_of_samples_per_label)
    label_with_minimum_number_of_samples = number_of_samples_per_label.index(
        minimum_number_of_samples_per_label
    )

    print("LABEL: ", label_with_minimum_number_of_samples)
    if n_shot + n_query < minimum_number_of_samples_per_label:
      flag = 1

    valid_dct: dict[int, list[int]] = {}
    for item, label in enumerate(valid_set.get_labels()):
      if label in valid_dct:
          valid_dct[label].append(item)
      else:
          valid_dct[label] = [item]
          
    number_of_samples_per_label = [
                len(items_for_label) for items_for_label in valid_dct.values()
            ]
    minimum_number_of_samples_per_label = min(number_of_samples_per_label)
    print("MIN: ", minimum_number_of_samples_per_label)
    label_with_minimum_number_of_samples = number_of_samples_per_label.index(
        minimum_number_of_samples_per_label
    )

    print("LABEL: ", label_with_minimum_number_of_samples)
    if n_shot + n_query > minimum_number_of_samples_per_label:
      flag = 0
    
    print("Step 1: ", flag, found)
    if flag == 1:
      found = 1
      break
  print("Step 2: ", flag, found)
  if found == 1: break
  found = 0

Folder /content/data/gtsrb/GTSRB/Training/00013 contains 90 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00037 contains 98 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00045 contains 74 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00053 contains 199 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00056 contains 95 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00022 contains 375 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00041 contains 148 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00028 contains 125 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00040 contains 242 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00001 contains 110 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00017 contains 79 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00019 contains 231 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00038 contains 285 .ppm files.
Folder /content/data/gtsrb/GTSRB/Training/00018 contains

In [23]:
target_val

50

In [ ]:
train_set.get_labels = lambda: [instance[1] for instance in train_set]

valid_set.get_labels = lambda: [instance[1] for instance in valid_set]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSamplerr(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
print("Done Train")
val_sampler = TaskSamplerr(
    valid_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

In [33]:
from torchvision.datasets import GTSRB
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100
image_size = 28

# # Instantiate the datasets
# train_set = GTSRB(
#     root="./data",
#     # background=True,
#     split="train",
#     transform=transforms.Compose(
#         [
#             transforms.Grayscale(num_output_channels=3),
#             transforms.RandomResizedCrop(image_size),
#             transforms.RandomHorizontalFlip(),
#             transforms.ToTensor(),
#         ]
#     ),
#     download=False,
# )

# train_set_size = int(len(train_set) * 0.8)
# val_set_size = len(train_set) - train_set_size
# train_set, valid_set = torch.utils.data.random_split(train_set, [train_set_size, val_set_size])

# train_set, valid_set = uniform_random_split(train_set)

# train_set.get_labels = lambda: [instance[1] for instance in train_set]

# valid_set.get_labels = lambda: [instance[1] for instance in valid_set]

# # Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
print("Done Train")
val_sampler = TaskSampler(
    valid_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

val_loader = DataLoader(
    valid_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

Done Train


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [35]:
len(valid_set)

643

In [40]:
l_lst = []
for i in range(len(valid_set)):
  l_lst.append(valid_set[i][1])

In [41]:
import numpy as np
print(len(np.unique(l_lst)))

17


And then we define the network. Here I chose Prototypical Networks and the built-in ResNet18 from PyTorch because it's easy.

In [ ]:
#Only used this encoder for RelationNetworks few shot algorithm

class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        #out = out.view(out.size(0),-1)
        return out # 64

In [48]:
from easyfsl.methods import RelationNetworks, FewShotClassifier
from easyfsl.modules import resnet12


convolutional_network = CNNEncoder()
few_shot_classifier = RelationNetworks(convolutional_network, feature_dimension = 64).to(DEVICE)

Now let's define our training helpers ! I chose to use Stochastic Gradient Descent on 200 epochs with a scheduler that divides the learning rate by 10 after 120 and 160 epochs. The strategy is derived from [this repo](https://github.com/fiveai/on-episodes-fsl).

We're also gonna use a TensorBoard because it's always good to see what your training curves look like.

In [49]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

And now let's get to it! Here we define the function that performs a training epoch.

We use tqdm to monitor the training in real time in our logs.

In [50]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

And we have everything we need! To perform validations we'll just use the built-in `evaluate` function from `easyfsl.methods.utils`.

This is now the time to **start training**.

I added something to log the state of the model that gave the best performance on the validation set.

In [52]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|██████████| 100/100 [00:05<00:00, 17.64it/s, accuracy=0.731]

Ding ding ding! We found a new best model!
Epoch 1



Validation: 100%|██████████| 100/100 [00:06<00:00, 15.95it/s, accuracy=0.77]

Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|██████████| 100/100 [00:06<00:00, 14.91it/s, accuracy=0.958]

Epoch 198



Validation: 100%|██████████| 100/100 [00:07<00:00, 13.06it/s, accuracy=0.934]

Epoch 199



Validation: 100%|██████████| 100/100 [00:08<00:00, 12.27it/s, accuracy=0.941]


Yay we successfully performed Episodic Training! Now if you want to you can retrieve the best model's state.

In [53]:
few_shot_classifier.load_state_dict(best_state)

<All keys matched successfully>

## Evaluation

Now that our model is trained, we want to test it.

First step: we fetch the test data.

In [54]:
import os

old_folder_name = '/content/data/gtsrb/GTSRB/Training'
new_folder_name = '/content/data/gtsrb/GTSRB/Trainingggg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/content/data/gtsrb/GTSRB/Trainingg'
new_folder_name = '/content/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

# For n_test_tasks = 1000

In [55]:
n_test_tasks = 1000

test_set = GTSRB(
    root="./data",
    # background=False,
    split="train",
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    ),
    download=False,
)

test_set.get_labels = lambda: [instance[1] for instance in test_set]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

Second step: we run the few-shot classifier on the test data.

In [56]:
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 1000/1000 [00:44<00:00, 22.60it/s, accuracy=0.761]

Average accuracy : 76.09 %


# For n_test_tasks = 10000

In [57]:
n_test_tasks = 10000

test_set = GTSRB(
    root="./data",
    # background=False,
    split="train",
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    ),
    download=False,
)

test_set.get_labels = lambda: [instance[1] for instance in test_set]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [58]:
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 10000/10000 [07:13<00:00, 23.08it/s, accuracy=0.759]

Average accuracy : 75.93 %


In [59]:
torch.save(few_shot_classifier, 'model.pth')

In [60]:
nahian = torch.load('model.pth')

In [61]:
few_shot_classifier.state_dict()

OrderedDict([('backbone.layer1.0.weight',
              tensor([[[[-6.3456e-02, -3.2317e-02, -1.0411e-01],
                        [ 8.6533e-02, -2.8199e-02, -1.3228e-01],
                        [ 1.9502e-01,  7.3431e-02, -7.6206e-02]],
              
                       [[-6.4705e-02, -2.8337e-02, -1.0290e-01],
                        [ 9.2724e-02, -3.2615e-02, -1.3101e-01],
                        [ 2.0591e-01,  8.1902e-02, -6.7469e-02]],
              
                       [[-6.6839e-02, -2.2988e-02, -9.2954e-02],
                        [ 9.5857e-02, -2.9953e-02, -1.2360e-01],
                        [ 2.0846e-01,  8.1817e-02, -6.5237e-02]]],
              
              
                      [[[-7.3603e-02, -1.1134e-01, -1.1691e-01],
                        [ 3.4300e-02,  1.6304e-02, -3.4418e-02],
                        [ 1.1333e-01,  1.1498e-01,  5.2232e-02]],
              
                       [[-6.5942e-02, -1.0008e-01, -1.2895e-01],
                        [ 3.3578e

In [62]:
few_shot_classifier.load_state_dict(few_shot_classifier.state_dict())

<All keys matched successfully>

In [63]:
train_optimizer.state_dict()

{'state': {0: {'momentum_buffer': tensor([[[[-0.0277, -0.0242, -0.0221],
             [-0.0255, -0.0236, -0.0190],
             [-0.0232, -0.0248, -0.0231]],
   
            [[-0.0277, -0.0242, -0.0221],
             [-0.0255, -0.0237, -0.0190],
             [-0.0231, -0.0248, -0.0231]],
   
            [[-0.0277, -0.0241, -0.0221],
             [-0.0255, -0.0237, -0.0190],
             [-0.0231, -0.0248, -0.0231]]],
   
   
           [[[-0.0530, -0.0573, -0.0631],
             [-0.0533, -0.0593, -0.0662],
             [-0.0587, -0.0630, -0.0691]],
   
            [[-0.0529, -0.0573, -0.0631],
             [-0.0533, -0.0593, -0.0662],
             [-0.0587, -0.0630, -0.0691]],
   
            [[-0.0529, -0.0573, -0.0631],
             [-0.0532, -0.0592, -0.0662],
             [-0.0587, -0.0630, -0.0692]]],
   
   
           [[[-0.0026, -0.0012, -0.0024],
             [-0.0048, -0.0025, -0.0029],
             [-0.0076, -0.0059, -0.0047]],
   
            [[-0.0027, -0.0012, -0.0025],


In [64]:
torch.save({
            'model_state_dict': few_shot_classifier.state_dict(),
            'optimizer_state_dict': train_optimizer.state_dict(),
            }, 'final.pth')

Congrats! You performed Episodic Training using EasyFSL. If you want to compare with a model trained using classical training, look at [this other example notebook](classical_training.ipynb).
